In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
df = pd.read_csv('./Data/winequality-white.csv', delimiter=';')

In [12]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [18]:
X_cols, y_col = [col for col in df.columns if col != 'quality'], 'quality'

In [19]:
X, y = df[X_cols], df[y_col]

In [26]:
X.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')

In [27]:
def standardize_data(X, y, test_size=0.30, SEED=0) -> tuple:
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import StandardScaler
    z = StandardScaler()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=SEED)
    Xz_train, Xz_test = z.fit_transform(X_train), z.transform(X_test)
    return (Xz_train, Xz_test, y_train, y_test)


In [28]:
Xz_train, Xz_test, y_train, y_test = standardize_data(X, y)

In [33]:
# showing x values are in z score form now
Xz_train[0]

array([-0.54987946,  0.10802059,  0.90421574, -0.54833003,  6.81327372,
        2.32630491,  1.04952979,  0.07318938, -1.17885395, -0.95161186,
       -1.16015744])